# TC2C for VISTA project - 

In [2]:
import os
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import matplotlib
import anndata

import scanpy as sc
import cell2cell as c2c

import liana as li
import plotnine as p9

from tqdm.auto import tqdm

from tqdm.auto import tqdm
from anndata import read_h5ad
import sys

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/pkgs/miniforge3/envs/CellPhoneDB/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/pkgs/miniforge3/envs/CellPhoneDB/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.


In [2]:
import warnings
warnings.filterwarnings('ignore')

## Creating the neutrophil dataset

In [2]:
#import the data : neutro annotations and chuvio data
# -> looking at where the data from before hand comes from ... 

In [16]:
print(f"neutro annot L2 : {len(neutro_annotations_L2_1)} / chuvio L2 {len(chuvio_L2_1)}\nneutro annot L3 : {len(neutro_annotations_L3_1)} / chuvio L3 {len(chuvio_L3_1)}")

neutro annot L2 : 37300 / chuvio L2 37033
neutro annot L3 : 244685 / chuvio L3 244685


In [14]:
chuvio_L3_1

AnnData object with n_obs × n_vars = 244685 × 311
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'libsize_drop', 'sizeFactor', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'singler_annotation', 'ident'
    uns: 'X_name'
    obsm: 'PCA', 'TSNE', 'UMAP'
    layers: 'logcounts'

In [39]:
#import the neutro annotations
neutro_annotations_L1_1 = pd.read_csv("/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/results/celltype_annotation/singler_db/L1_1_neutrophils_l3.csv")
neutro_annotations_L1_2 = pd.read_csv("/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/results/celltype_annotation/singler_db/L1_2_neutrophils_l3.csv")
neutro_annotations_L2_1 = pd.read_csv("/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/results/celltype_annotation/singler_db/L2_1_neutrophils_l3.csv")
neutro_annotations_L3_1 = pd.read_csv("/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/results/celltype_annotation/singler_db/L3_1_neutrophils_l3.csv")
neutro_annotations_L4_1 = pd.read_csv("/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/results/celltype_annotation/singler_db/L4_1_neutrophils_l3.csv")

#set the index as the name of the cell 
neutro_annotations_L1_1.set_index(neutro_annotations_L1_1.columns[0], inplace=True)
neutro_annotations_L1_2.set_index(neutro_annotations_L1_2.columns[0], inplace=True)
neutro_annotations_L2_1.set_index(neutro_annotations_L2_1.columns[0], inplace=True)
neutro_annotations_L3_1.set_index(neutro_annotations_L3_1.columns[0], inplace=True)
neutro_annotations_L4_1.set_index(neutro_annotations_L4_1.columns[0], inplace=True)

#modify the neutro annotations index -> exchanged the named of the indices to make up for the wrong axis in those fields. 
neutro_annotations_L1_1.index = "L1_1_" + neutro_annotations_L1_1.index
neutro_annotations_L1_2.index = "L1_2_" + neutro_annotations_L1_2.index
neutro_annotations_L2_1.index = "L3_1_" + neutro_annotations_L2_1.index
neutro_annotations_L3_1.index = "L2_1_" + neutro_annotations_L3_1.index
neutro_annotations_L4_1.index = "L4_1_" + neutro_annotations_L4_1.index

#Reading in the original anndata reads : 
path_chuvio_L1_1 = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/xenium_objs/anndata/chuvio_L1_1.h5ad"
chuvio_L1_1 = sc.read_h5ad(path_chuvio_L1_1)

path_chuvio_L1_2 = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/xenium_objs/anndata/chuvio_L1_2.h5ad"
chuvio_L1_2 = sc.read_h5ad(path_chuvio_L1_2)

path_chuvio_L2_1 = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/xenium_objs/anndata/chuvio_L2_1.h5ad"
chuvio_L2_1 = sc.read_h5ad(path_chuvio_L2_1)

path_chuvio_L3_1 = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/xenium_objs/anndata/chuvio_L3_1.h5ad"
chuvio_L3_1 = sc.read_h5ad(path_chuvio_L3_1)

path_chuvio_L4_1 = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium/xenium_objs/anndata/chuvio_L4_1.h5ad"
chuvio_L4_1 = sc.read_h5ad(path_chuvio_L4_1)

In [17]:
chuvio_L2_1.obs

,orig.ident,nCount_originalexp,nFeature_originalexp,cell_id,x_centroid,y_centroid,transcript_counts,control_probe_counts,control_codeword_counts,unassigned_codeword_counts,...,subsets_Mito_percent,total,libsize_drop,sizeFactor,nCount_SCT,nFeature_SCT,SCT_snn_res.0.8,seurat_clusters,singler_annotation,ident
L3_1_aaadaibf-1,L3,16.0,16,L3_1_aaadaibf-1,1386.738037,2533.420166,16,0,0,0,...,0.0,16.0,False,0.109692,76.0,42,6,6,Smooth_muscle,6
L3_1_aaadkmaf-1,L3,23.0,23,L3_1_aaadkmaf-1,3297.584229,863.205688,23,0,0,0,...,0.0,23.0,False,0.157683,76.0,42,1,1,DC_2,1
L3_1_aaafjbme-1,L3,46.0,32,L3_1_aaafjbme-1,3354.973633,863.813904,46,0,0,0,...,0.0,46.0,False,0.315365,95.0,33,6,6,Lung_epithelia,6
L3_1_aaagppme-1,L3,61.0,52,L3_1_aaagppme-1,3295.632324,675.669495,61,0,0,0,...,0.0,61.0,False,0.418201,99.0,52,1,1,Pericytes,1
L3_1_aaajnbae-1,L3,27.0,22,L3_1_aaajnbae-1,3306.216309,845.382568,27,0,0,0,...,0.0,27.0,False,0.185106,79.0,39,1,1,Lung_epithelia,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
L3_1_oipabjcp-1,L3,169.0,63,L3_1_oipabjcp-1,1215.482056,1754.516113,169,0,0,0,...,0.0,169.0,False,1.158624,133.0,63,12,12,Mast cells,12
L3_1_oipafihf-1,L3,139.0,54,L3_1_oipafihf-1,1222.162598,1751.131470,139,0,0,0,...,0.0,139.0,False,0.952951,126.0,54,4,4,DC_2,4
L3_1_oipagoio-1,L3,136.0,59,L3_1_oipagoio-1,1221.880005,1768.429199,136,0,0,0,...,0.0,136.0,False,0.932384,128.0,59,12,12,Macrophages,12
L3_1_oipahcba-1,L3,23.0,17,L3_1_oipahcba-1,2828.983154,3401.302490,23,0,0,0,...,0.0,23.0,False,0.157683,88.0,34,2,2,Mast cells,2


In [18]:
#before we create the new versions of these databases we want to check that they both have the same numbers of cells

# Get the set of index values for neutro_annotations
neutro_annotations_L1_1_set = set(neutro_annotations_L1_2.index)

# Get the set of index values for L4_1 observation metadata
chuvio_L1_1_index_set = set(chuvio_L1_2.obs.index)

# Find the intersection of the two sets
common_names = neutro_annotations_L1_1_set.intersection(chuvio_L1_1_index_set)

# Get the number of common names
num_common_names = len(common_names)

# Print the number of common names
print("Number of common names:", num_common_names)



Number of common names: 53964


In [40]:
#filter our from the chuvio databases the cells that aren't in the neutro_VISTA version ones

# Get the set of indices from neutro_annotations
neutro_annotations_L1_1_indices = set(neutro_annotations_L1_1.index)
neutro_annotations_L1_2_indices = set(neutro_annotations_L1_2.index)
neutro_annotations_L2_1_indices = set(neutro_annotations_L2_1.index)
neutro_annotations_L3_1_indices = set(neutro_annotations_L3_1.index)
neutro_annotations_L4_1_indices = set(neutro_annotations_L4_1.index)

# Filter cells in chuvio_L4_1 to keep only those present in neutro_annotations
chuvio_L1_1_filtered = chuvio_L1_1[chuvio_L1_1.obs.index.isin(neutro_annotations_L1_1.index)]
chuvio_L1_2_filtered = chuvio_L1_2[chuvio_L1_2.obs.index.isin(neutro_annotations_L1_2.index)]
chuvio_L2_1_filtered = chuvio_L2_1[chuvio_L2_1.obs.index.isin(neutro_annotations_L2_1.index)]
chuvio_L3_1_filtered = chuvio_L3_1[chuvio_L3_1.obs.index.isin(neutro_annotations_L3_1.index)]
chuvio_L4_1_filtered = chuvio_L4_1[chuvio_L4_1.obs.index.isin(neutro_annotations_L4_1.index)]

In [53]:
#now creating the new versions of these databases ... 

chuvio_L1_1_filtered.obs['new_VISTA_annot'] = neutro_annotations_L1_1['neutro_annot']
chuvio_L1_2_filtered.obs['new_VISTA_annot'] = neutro_annotations_L1_2['neutro_annot']
chuvio_L2_1_filtered.obs['new_VISTA_annot'] = neutro_annotations_L2_1['neutro_annot']
chuvio_L3_1_filtered.obs['new_VISTA_annot'] = neutro_annotations_L3_1['neutro_annot']
chuvio_L4_1_filtered.obs['new_VISTA_annot'] = neutro_annotations_L4_1['neutro_annot']

In [71]:
chuvio_L1_1_filtered

AnnData object with n_obs × n_vars = 71187 × 313
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'libsize_drop', 'sizeFactor', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'singler_annotation', 'ident', 'new_VISTA_annot'
    uns: 'X_name'
    obsm: 'PCA', 'TSNE', 'UMAP'
    layers: 'logcounts'

In [72]:
chuvio_L1_2_filtered

AnnData object with n_obs × n_vars = 53964 × 318
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'libsize_drop', 'sizeFactor', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'singler_annotation', 'ident', 'new_VISTA_annot'
    uns: 'X_name'
    obsm: 'PCA', 'TSNE', 'UMAP'
    layers: 'logcounts'

In [74]:
chuvio_L1_1_filtered

AnnData object with n_obs × n_vars = 71187 × 313
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'libsize_drop', 'sizeFactor', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'singler_annotation', 'ident', 'new_VISTA_annot'
    uns: 'X_name'
    obsm: 'PCA', 'TSNE', 'UMAP'
    layers: 'logcounts'

In [75]:
#Save the object
chuvio_L1_1_VISTA = chuvio_L1_1_filtered
chuvio_L1_1_VISTA.write_h5ad('data/chuvio_L1_1_VISTA.h5ad')

chuvio_L1_2_VISTA = chuvio_L1_2_filtered
chuvio_L1_2_VISTA.write_h5ad('data/chuvio_L1_2_VISTA.h5ad')

chuvio_L2_1_VISTA = chuvio_L2_1_filtered
chuvio_L2_1_VISTA.write_h5ad('data/chuvio_L2_1_VISTA.h5ad')

chuvio_L3_1_VISTA = chuvio_L3_1_filtered
chuvio_L3_1_VISTA.write_h5ad('data/chuvio_L3_1_VISTA.h5ad')

chuvio_L4_1_VISTA = chuvio_L4_1_filtered
chuvio_L4_1_VISTA.write_h5ad('data/chuvio_L4_1_VISTA.h5ad')

## 1 Inital Setup

In [58]:
data_folder = '/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astearAnalysis/TensorC2C/data/liana-outputs/'
output_folder = '/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/Analysis/TensorC2C/data/tc2c-outputs/'
c2c.io.directories.create_directory(output_folder)

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/Analysis/TensorC2C/data/tc2c-outputs/ already exists.


In [3]:
# Import the modified andata sets. 
path = '/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/Analysis/TensorC2C/data/'
chuvio_L1_1_VISTA = anndata.read_h5ad(f"{path}chuvio_L1_1_VISTA.h5ad")
chuvio_L1_2_VISTA = anndata.read_h5ad(f"{path}chuvio_L1_2_VISTA.h5ad")
chuvio_L2_1_VISTA = anndata.read_h5ad(f"{path}chuvio_L2_1_VISTA.h5ad")
chuvio_L3_1_VISTA = anndata.read_h5ad(f"{path}chuvio_L3_1_VISTA.h5ad")
chuvio_L4_1_VISTA = anndata.read_h5ad(f"{path}chuvio_L4_1_VISTA.h5ad")


In [4]:
chuvio_L1_1_VISTA["singler_annotation"]

KeyError: 'singler_annotation'

In [ ]:
#Che

In [82]:
#need to concatenate the whole dataset, renamed as adata. 
adata = anndata.concat([chuvio_L1_1_VISTA, chuvio_L1_2_VISTA, chuvio_L2_1_VISTA, chuvio_L3_1_VISTA, chuvio_L4_1_VISTA])

In [93]:
#Need to create a sample column to keep track of each ... (should be coding the context before running Liana ?? )
sample_names = [index[:4] for index in adata.obs.index]
adata.obs['sample'] = sample_names

In [94]:
li.mt.rank_aggregate.by_sample(adata,
                                   sample_key='sample',
                                   groupby='new_VISTA_annot',
                                   resource_name = 'consensus',
                                   expr_prop=0.1, # must be expressed in expr_prop fraction of cells
                                   min_cells = 5,
                                   n_perms = 100,
                                   use_raw = False, # run on log- and library-normalized counts
                                   verbose = True,
                                   inplace = True
                                  )

Converting `sample` to categorical!
Now running: L4_1: 100%|██████████| 5/5 [05:11<00:00, 62.22s/it]


In [104]:
adata.uns['liana_res']

,sample,source,target,ligand_complex,receptor_complex,lr_means,cellphone_pvals,expr_prod,scaled_weight,lr_logfc,spec_weight,lrscore,lr_probs,cellchat_pvals,specificity_rank,magnitude_rank
0,L1_1,Endothelia_vascular,Tu_L1,VIM,CD44,8.660626,0.00,66.489204,0.816888,6.821331,0.008928,0.936598,0.003462,0.0,0.004413,1.455201e-11
1,L1_1,Pericyte,Tu_L1,VIM,CD44,7.369877,0.00,51.665741,0.609583,10.340343,0.006937,0.928684,0.002536,0.0,0.006740,6.062811e-10
2,L1_1,Endothelia_vascular,DC_activated,VIM,CD44,7.505217,0.00,39.732101,0.514105,-45.493233,0.005335,0.919482,0.002146,0.0,0.173988,4.989164e-09
3,L1_1,Endothelia_vascular,T_CXCL13,VIM,CD44,7.456196,0.00,38.596867,0.501257,-47.006069,0.005182,0.918402,0.002064,0.0,0.185154,1.454250e-08
4,L1_1,Macrophage,Tu_L1,VIM,CD44,7.020940,0.00,47.658405,0.553547,21.422409,0.006399,0.925963,0.002198,0.0,0.010930,1.636134e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87247,L4_1,no_assigned_label,T_CXCL13,CXCL12,CD4,0.405503,0.00,0.160719,0.106499,-16.364040,0.002320,0.450343,0.000000,1.0,NaN,1.000000e+00
87248,L4_1,T_CD4,DC_2,SPP1,CD44,1.387094,0.55,0.608303,-0.005695,-inf,0.001381,0.614489,0.000000,1.0,NaN,1.000000e+00
87249,L4_1,T_CTL,Neutrophil_VISR_pos,HGF,CD44,1.437197,0.00,0.368700,0.041948,-inf,0.001717,0.553761,0.000000,1.0,NaN,1.000000e+00
87250,L4_1,T_CTL,Neutrophil_VISR_pos,HLA-A,ERBB2,0.784068,1.00,0.550868,-0.033964,-38.719383,0.001217,0.602677,0.000000,1.0,NaN,1.000000e+00


In [105]:
liana_res = adata.uns['liana_res']

In [108]:
#saved resulting object
adata_VISTA_Liana = adata
adata_VISTA_Liana.write_h5ad("adata_VISTA_Liana.h5ad")

## 3.2 Create 4D-Communication Tensor

Specifying the context. In the original they have 12 samples and the first letter tells them if it is Control or sick, so the following ordering operation allows it to setup the order of samples depending on the context. We jsut want to see the dfference between L4 that has two LSE and the others. 

In [109]:
sorted_samples = sorted(liana_res['sample'].unique())

In [110]:
liana_res

,sample,source,target,ligand_complex,receptor_complex,lr_means,cellphone_pvals,expr_prod,scaled_weight,lr_logfc,spec_weight,lrscore,lr_probs,cellchat_pvals,specificity_rank,magnitude_rank
0,L1_1,Endothelia_vascular,Tu_L1,VIM,CD44,8.660626,0.00,66.489204,0.816888,6.821331,0.008928,0.936598,0.003462,0.0,0.004413,1.455201e-11
1,L1_1,Pericyte,Tu_L1,VIM,CD44,7.369877,0.00,51.665741,0.609583,10.340343,0.006937,0.928684,0.002536,0.0,0.006740,6.062811e-10
2,L1_1,Endothelia_vascular,DC_activated,VIM,CD44,7.505217,0.00,39.732101,0.514105,-45.493233,0.005335,0.919482,0.002146,0.0,0.173988,4.989164e-09
3,L1_1,Endothelia_vascular,T_CXCL13,VIM,CD44,7.456196,0.00,38.596867,0.501257,-47.006069,0.005182,0.918402,0.002064,0.0,0.185154,1.454250e-08
4,L1_1,Macrophage,Tu_L1,VIM,CD44,7.020940,0.00,47.658405,0.553547,21.422409,0.006399,0.925963,0.002198,0.0,0.010930,1.636134e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87247,L4_1,no_assigned_label,T_CXCL13,CXCL12,CD4,0.405503,0.00,0.160719,0.106499,-16.364040,0.002320,0.450343,0.000000,1.0,NaN,1.000000e+00
87248,L4_1,T_CD4,DC_2,SPP1,CD44,1.387094,0.55,0.608303,-0.005695,-inf,0.001381,0.614489,0.000000,1.0,NaN,1.000000e+00
87249,L4_1,T_CTL,Neutrophil_VISR_pos,HGF,CD44,1.437197,0.00,0.368700,0.041948,-inf,0.001717,0.553761,0.000000,1.0,NaN,1.000000e+00
87250,L4_1,T_CTL,Neutrophil_VISR_pos,HLA-A,ERBB2,0.784068,1.00,0.550868,-0.033964,-38.719383,0.001217,0.602677,0.000000,1.0,NaN,1.000000e+00


## 3.3 Generate tensor

To generate the 4D-communication tensor, we will to create matrices with the communication scores for each of the ligand-receptor pairs within the same sample, then generate a 3D tensor for each sample, and finally concatenate them to form the 4D tensor.

Briefly, we use the LIANA dataframe and communication scores to organize them as follows:
-> If you used a pipeline that generated negative scores, replace these with 0,  Otherwise, by default, Tensor-cell2cell will treat these as NaN.
-> ...

In [111]:
#Here we use the magnitude score but we could use the specificity score, in score_key. Also try lowering the outer_fraction value for rare occurences
tensor = li.multi.to_tensor_c2c(liana_res=liana_res, # LIANA's dataframe containing results
                                sample_key='sample', # Column name of the samples
                                source_key='source', # Column name of the sender cells
                                target_key='target', # Column name of the receiver cells
                                ligand_key='ligand_complex', # Column name of the ligands
                                receptor_key='receptor_complex', # Column name of the receptors
                                score_key='magnitude_rank', # Column name of the communication scores to use
                                non_negative = True, # set negative values to 0
                                inverse_fun=lambda x: 1 - x, # Transformation function
                                non_expressed_fill=None, # Value to replace missing values with
                                how='outer', # What to include across all samples
                                lr_fill=np.nan, # What to fill missing LRs with
                                cell_fill = np.nan, # What to fill missing cell types with
                                outer_fraction=1/10., # Fraction of samples as threshold to include cells and LR pairs.
                                lr_sep='^', # How to separate ligand and receptor names to name LR pair
                                context_order=sorted_samples, # Order to store the contexts in the tensor
                                sort_elements=True # Whether sorting alphabetically element names of each tensor dim. Does not apply for context order if context_order is passed.
                               )

100%|██████████| 5/5 [00:01<00:00,  3.91it/s]


## 3.5 Prepare Tensor Metadata -> Selecting the context

In [95]:
adata.obs

,orig.ident,nCount_originalexp,nFeature_originalexp,cell_id,x_centroid,y_centroid,transcript_counts,control_probe_counts,control_codeword_counts,unassigned_codeword_counts,...,libsize_drop,sizeFactor,nCount_SCT,nFeature_SCT,SCT_snn_res.0.8,seurat_clusters,singler_annotation,ident,new_VISTA_annot,sample
L1_1_aaaadbga-1,L1,413.0,103,L1_1_aaaadbga-1,1186.546265,1686.810181,413,1,0,0,...,False,2.515645,143.0,52,9,9,Fibroblasts,9,Fibroblast,L1_1
L1_1_aaaajnma-1,L1,253.0,86,L1_1_aaaajnma-1,1194.922241,1699.988892,253,1,0,0,...,False,1.541061,159.0,78,9,9,Fibroblasts,9,Pericyte,L1_1
L1_1_aaadecno-1,L1,176.0,66,L1_1_aaadecno-1,1196.921997,1708.868652,176,0,0,0,...,False,1.072042,157.0,66,9,9,Pericytes,9,Pericyte,L1_1
L1_1_aaafkiae-1,L1,293.0,111,L1_1_aaafkiae-1,1204.801636,1698.712280,293,1,0,0,...,False,1.784707,155.0,88,6,6,Fibroblasts,6,Fibroblast,L1_1
L1_1_aaahijio-1,L1,175.0,73,L1_1_aaahijio-1,1209.148804,1690.650635,175,0,0,1,...,False,1.065951,154.0,72,8,8,Pericytes,8,Pericyte,L1_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
L4_1_oilnnjie-1,L4,162.0,77,L4_1_oilnnjie-1,7791.851562,5969.610352,165,1,0,0,...,False,1.107579,123.0,77,16,16,Granulocytes,16,no_assigned_label,L4_1
L4_1_oilnoiin-1,L4,157.0,70,L4_1_oilnoiin-1,7790.732422,5962.326172,157,0,0,1,...,False,1.073395,123.0,70,15,15,DC_1,15,no_assigned_label,L4_1
L4_1_oiloajgk-1,L4,198.0,78,L4_1_oiloajgk-1,7784.906250,5955.801270,198,1,0,0,...,False,1.353708,128.0,73,3,3,Granulocytes,3,no_assigned_label,L4_1
L4_1_oilocpdb-1,L4,57.0,39,L4_1_oilocpdb-1,7779.752930,5969.401855,57,0,0,0,...,False,0.389704,101.0,39,2,2,DC_1,2,no_assigned_label,L4_1


In [112]:
from collections import defaultdict

element_dict = defaultdict(lambda: 'Unknown')

In [113]:
#orig.ident isn't exactly the same as the one in sorted sample, might cause problems later on, so keep an eye out for it. 
context_dict = adata.obs.set_index('sample')['orig.ident'].sort_values().to_dict()

In [114]:
context_dict = element_dict.copy()

In [115]:
sorted_samples

['L1_1', 'L1_2', 'L2_1', 'L3_1', 'L4_1']

In [116]:
context_dict.update({'adata_L1_1_LT' : 'L1',
                     'adata_L2_2_LT' : 'L2',
                     'adata_L2_2_rep_LT' : 'L2',
                     'adata_L3_1_LT' : 'L3',
                     'adata_L4_1_LT' : 'L4',
                     })
dimensions_dict = [context_dict, None, None, None]

In [117]:
meta_tensor = c2c.tensor.generate_tensor_metadata(interaction_tensor=tensor,
                                              metadata_dicts=[context_dict, None, None, None],
                                              fill_with_order_elements=True
                                             )

In [118]:
pd.__version__

'2.2.0'

## 3.6 Exporting tensors

In [121]:
c2c.io.export_variable_with_pickle(tensor, output_folder + 'BALF-Tensor-5chuvio-VISTA_15_04_24.pkl')

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/Analysis/TensorC2C/data/tc2c-outputs/BALF-Tensor-5chuvio-VISTA_15_04_24.pkl  was correctly saved.


In [122]:
c2c.io.export_variable_with_pickle(meta_tensor, output_folder + 'BALF-Tensor-5chuvio-VISTA-Metadata_15_04_24.pkl')

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/Analysis/TensorC2C/data/tc2c-outputs/BALF-Tensor-5chuvio-VISTA-Metadata_15_04_24.pkl  was correctly saved.
